In [2]:
import os
import pandas as pd
import zstandard as zstd

def decompress_zst_to_csv(zst_file, output_csv):
    """Decompresses a .zst file into a .csv file."""
    with open(zst_file, 'rb') as f:
        dctx = zstd.ZstdDecompressor()
        with open(output_csv, 'wb') as out_f:
            out_f.write(dctx.decompress(f.read()))

def combine_csv_zst_files(input_dir, output_file):
    """
    Combines multiple .csv.zst files in a directory into a single .csv.zst file.
    
    Args:
        input_dir (str): Directory containing the .csv.zst files.
        output_file (str): Path to the combined output .csv.zst file.
    """
    temp_dir = "temp_csv_files"
    os.makedirs(temp_dir, exist_ok=True)
    
    combined_df = pd.DataFrame()
    
    # Decompress each .zst file and read the CSV
    for zst_file in os.listdir(input_dir):
        if zst_file.endswith('.csv.zst'):
            temp_csv = os.path.join(temp_dir, zst_file.replace('.zst', ''))
            decompress_zst_to_csv(os.path.join(input_dir, zst_file), temp_csv)
            df = pd.read_csv(temp_csv)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Save the combined DataFrame to a .csv file
    combined_csv = "combined.csv"
    combined_df.to_csv(combined_csv, index=False)
    
    # Compress the combined .csv to .zst
    with open(combined_csv, 'rb') as f:
        cctx = zstd.ZstdCompressor(level=3)
        with open(output_file, 'wb') as out_f:
            out_f.write(cctx.compress(f.read()))
    
    # Clean up temporary files
    os.remove(combined_csv)
    for temp_file in os.listdir(temp_dir):
        os.remove(os.path.join(temp_dir, temp_file))
    os.rmdir(temp_dir)

# Example usage:
# Provide the directory containing your .csv.zst files and the output file name
input_directory = "/Users/glennfor/Downloads/options"
output_combined_file = "combined_output.csv.zst"

combine_csv_zst_files(input_directory, output_combined_file)


PermissionError: [Errno 1] Operation not permitted: '/Users/glennfor/Downloads/options'